In [ ]:
from uuid import UUID
import sys
import logging 
import arrow
import pandas as pd
import public_data

import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.core.get_database as edb
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt

logging.getLogger().setLevel(logging.DEBUG)
logger = logging.getLogger()
#logger.handlers[0].stream = sys.stdout

In [ ]:
#android_ids = ["70968068-dba5-406c-8e26-09b548da0e4b", "6561431f-d4c1-4e0f-9489-ab1190341fb7",
#             "92cf5840-af59-400c-ab72-bab3dcdf7818", "93e8a1cc-321f-4fa9-8c3c-46928668e45d"]
# nexus
iphone_ids = ["079e0f1a-c440-3d7c-b0e7-de160f748e35", "95e70727-a04e-3e33-b7fe-34ab19194f8b"]
# Moto
android_ids = [ "fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7", "3bc0f91f-7660-34a2-b005-5c399598a369"]

In [ ]:
# pull data from the server 
server_url = 'https://e-mission.eecs.berkeley.edu'

#overall
start_ts = arrow.get('2016-10-18T20:00:00-07:00').timestamp #time in UTC
end_ts = arrow.get('2016-10-19T03:00:00-07:00').timestamp#time in UTC
print start_ts, end_ts
phone_ids = iphone_ids+android_ids

public_data.request_batched_data(server_url, start_ts, end_ts, phone_ids)

In [ ]:
logger.setLevel(logging.CRITICAL)

android_ts = map(lambda u: esta.TimeSeries.get_time_series(UUID(u)), android_ids)
#time query specification
time_range = estt.TimeQuery("metadata.write_ts", start_ts, end_ts)
android_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), android_ts)
android_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), android_ts)


iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(UUID(u)), iphone_ids)
#time query specification
ios_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), iphone_ts)
iphone_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), iphone_ts)


In [ ]:
# iphone_test = iphone_battery_df_list[0]
# iphone_test[['fmt_time', 'battery_level_pct', 'battery_status']]

In [ ]:
import numpy as np

android_xnew = np.arange(start_ts, end_ts, 15 * 60) # 15 minutes
# android_xnew2 = np.arange(start_ts2, end_ts2, 15 * 60) # 15 minutes

In [ ]:
import scipy.interpolate as spi
import numpy as np

def get_interpolated(x, y, xnew):
    # fn = spi.interp1d(x=x, y=y, bounds_error=False, fill_value=y.iloc[-1])
    fn = spi.interp1d(x=x, y=y, bounds_error=False)
    return fn(xnew)
def get_interpolated_df(sensor_df, xnew):
    local_dt_new = map(pydt.datetime.fromtimestamp, xnew)
    fmt_time_new = map(lambda dt: dt.isoformat(), local_dt_new)
    interpolated_series = get_interpolated(sensor_df.ts, sensor_df.battery_level_pct, xnew)
    interpolated_df = pd.DataFrame(data={'ts': xnew, 'metadata_write_ts': xnew, 'local_dt': local_dt_new, 'fmt_time': fmt_time_new, 'battery_level_pct': interpolated_series})
    return interpolated_df

In [ ]:
android_battery_interp_df_list = map(lambda df: get_interpolated_df(df, android_xnew), android_battery_df_list)
iphone_battery_interp_df_list = map(lambda df: get_interpolated_df(df, android_xnew), iphone_battery_df_list)

In [ ]:
ig_start_ts = arrow.get('2016-10-17T18:00:00-07:00').timestamp #time in UTC
ig_end_ts = arrow.get('2016-10-17T20:30:00-07:00').timestamp#time in UTC

In [ ]:
sel_android_battery_df_list = map(lambda df: df[np.logical_not(np.logical_and(df.ts > ig_start_ts, df.ts < ig_end_ts))], android_battery_interp_df_list)
sel_iphone_battery_df_list = map(lambda df: df[np.logical_not(np.logical_and(df.ts > ig_start_ts, df.ts < ig_end_ts))], iphone_battery_interp_df_list)

In [ ]:
title = "variable workload (ios and android)"
text = "1hr,1hr,1hr,1hr"
label = ['"Nexus6"-1', '"Nexus6"-2']
label2 = ['"iPhone6"-1', '"iPhone6"-2']
#style = ['ob', 'og', 'or', 'oc']

style_interp = ['b', 'g']
style = ['ob', 'og']

style_interp2 = ['r', 'c']
style2 = ['or', 'oc']

ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 18})

In [ ]:
#Android: With interplation on battery data 

(ifig, iaxes2d) = plt.subplots(ncols=1, nrows=1, figsize=(8,4))
iaxes = iaxes2d

#ifig.suptitle(title + ' (Android_overlaid)', fontsize=14, fontweight='bold')

for i, df in enumerate(android_battery_interp_df_list):
    print "About to plot %d" % i
    df['hr'] = (df.ts-start_ts)/3600.0
    df['min'] = (df.ts-start_ts)/60.0
    df.plot(x='min', y='battery_level_pct', style=style_interp[i], ax=iaxes, label= label[i], legend=True)


for i, df in enumerate(android_battery_df_list):
    print "About to plot %d" % i
    df['hr'] = (df.ts-start_ts)/3600.0
    df['min'] = (df.ts-start_ts)/60.0
    df.plot(x='min', y='battery_level_pct', style=style[i], ax=iaxes,  xlim=(min(df.hr), max(df.hr)), legend=False)
    iaxes.set_xlabel('Minutes')
    iaxes.set_ylabel('Battery Level')
    #plt.xticks(np.arange(min(df.dt), max(df.dt), 1))
    

for i, df in enumerate(iphone_battery_interp_df_list):
    print "About to plot %d" % i
    df['hr'] = (df.ts-start_ts)/3600.0
    df['min'] = (df.ts-start_ts)/60.0
    df.plot(x='min', y='battery_level_pct', style=style_interp2[i], ax=iaxes, label=label2[i], legend=True)


for i, df in enumerate(iphone_battery_df_list):
    print "About to plot %d" % i
    df['hr'] = (df.ts-start_ts)/3600.0
    df['min'] = (df.ts-start_ts)/60.0
    df.plot(x='min', y='battery_level_pct', style=style2[i], ax=iaxes,  xlim=(min(df.hr), max(df.hr)), legend=False)
    
iaxes.set_xlabel('Minutes')
iaxes.set_ylabel('Battery % Level')
iaxes.vlines([120,240], 0, 100)
plt.xticks(np.arange(min(df['min']), max(df['min']), 30))

ifig.savefig('../graphs/parallel_validity_variable_workload_slow.pdf')